## Notebook: Contextual Anomaly Detection (CAD) Model Deployment

This is the second of a series of three notebooks that show how the Contextual Anomaly Detection (CAD) Accelerator can be used to train and deploy a prediction interval model into Monitor using the Model Factory service endpoints.
1.   Training: cookbooks/contextual_anomaly_train.ipynb
2.  Monitor Device Creation: cookbooks/contextual_anomaly_create_device.ipynb
3.   Model Deployment: cookbooks/contextual_anomaly_deploy_model.ipynb


### CAD Description

The CAD training job produces three prediction interval models that capture the normal operation (non-anomalous) behaviour of a given target variable based on a set of input features. 

These are point estimate multivariate regression machine learning models (denoted as base regressor) enhanced with conformal prediction statistical wrappers to produce a lower and upper bound that contain the target variable with probability 95% under normal operation (non-anomalous) conditions. Therefore, the probability of observing a target variable outside of the provided interval is 5% under normal operation conditions.

### Wind Turbine Dataset Description: 

In this notebook we use the CAD Accelerator to learn a prediction interval model that covers the normal operation behaviour of the Average Reactive Power of a Wind turbine asset with probability 95%. We consider 4 input features to predict the target variable. These are Average Active Power, Average Generator Bearing 1 Temperature, Average Generator Bearing 2 Temperature, Average Wind Speed.

### Model Deployment Notebook Description

This notebook shows an example of how we can deploy a trained CAD model into Monitor using the Model Factory Service. The deployed model will be configured as a streaming metric in the corresponding Monitor device.

This is the THIRD notebook of our tutorial and should be run after 'cookbooks/contextual_anomaly_train.ipynb' and   'cookbooks/contextual_anomaly_create_device.ipynb' since it requires the model specification file saved at the end of the CAD training notebook and the corresponding Monitor Device created with the contextual_anomaly_create_device notebook.





<a id='notebook_workflow'></a>
### Notebook Workflow
- [Imports](#imports)
- [Load Model Factory config file (yaml)](#load_mfyaml)
- [Load Model specification file (yaml)](#load_modelspecyaml)
- [Prepare Payload to Deploy Trained Model](#prepare_payload)
- [Post Model Deployment Job](#deploy_model_post)
- [Request job log and summary](#job_log)

<a id='imports'></a>
### Imports

In [1]:
import requests
import yaml
import pandas as pd

<a id='load_mfyaml'></a>
### Load Model Factory config file (yaml)


This file is a dictionary containing:

    - endpoint_url: <ACTION: Replace with Model Factory endpoint URL>
    - train_recipe_endpoint: recipe/supervised-anomaly (DON'T CHANGE)
    - deploy_recipe_endpoint: deployment/monitor/model/create (DON'T CHANGE)
    - create_device_recipe_endpoint: deployment/monitor/device/create (DON'T CHANGE)

In [2]:
model_factory_config_file_name = "../config/model_factory_config.yml"


with open(model_factory_config_file_name, 'r') as file:
    model_factory_config = yaml.safe_load(file)

print(model_factory_config)

{'endpoint_url': 'http://127.0.0.1:8000/ibm/modelfactory/service/', 'train_recipe_endpoint': 'recipe/supervised-anomaly', 'deploy_recipe_endpoint': 'deployment/monitor/model/create', 'create_device_recipe_endpoint': 'deployment/monitor/device/create'}


<a id='load_modelspecyaml'></a>
### Load Model specification file (yaml)

This is the .yaml file saved in the last step of the contextual_anomaly_train.ipynb notebook. It is a dictionary containing the following information:

    - onnx_model_uri : Model mlflow uri provided in the CAD training summary
    - train_job_id : Model Factory CAD training job id
    - mas_device_name : Name of the Monitor Device

In [3]:
input_file_name = "../config/model_info.yml"

with open(input_file_name, 'r') as file:
    model_info_data = yaml.safe_load(file)

print(model_info_data)

{'mas_device_name': 'Wind_Turbine_Test_1', 'onnx_model_uri': 's3://testdataupload/38/2d3ee3fa6c0f4d80b0c6f22b379cf993/artifacts/PI_model_prefit_LGBM1_best.onnx', 'train_job_id': '7151b7c4-9ce1-477d-9792-2ab3569d4593'}


<a id='prepare_payload'></a>
### Prepare Payload to Deploy Trained Model

The payload to deploy model into Monitor requires the following files:

- iot_credentials: This file is located in config/iot_credentials.yaml and is a dictionary containing the following information
    - IOT_URL: <INSERT>
    - IOT_API_KEY: <INSERT>
    - IOT_API_TOKEN: <INSERT>
    - model_api: /api/v0002/pipeline/models
    - MAS_URL: <INSERT>
    - X-api-key: <INSERT>
    - X-api-token: <INSERT>
    - mam_user_email: <INSERT>
    - tenantId: <INSERT>


- mas_device_name: Name of mas device where the model should be deployed (this will be obtained from the Model Specification file, [see above](#load_modelspecyaml) )

- onnx_model_uri: mlflow uri of the onnx model to be deployed (this will be obtained from the Model Specification file, [see above](#load_modelspecyaml) )

- train_job_id: CAD training job id of the model that is being deployed (this will be obtained from the Model Specification file, [see above](#load_modelspecyaml) )

- prepare_kpi_dashboard: Boolean indicating if a dashboard should be created

In [4]:
'''
Credential File Path and Name
'''

credentials_file_path = "../config/iot_credentials_beta.yaml"
credentials_file_name = "iot_credentials_beta.yaml"


'''
Device Name, ONNX Model URI, Train Job ID 
'''

mas_device_name = model_info_data['mas_device_name']
onnx_model_uri = model_info_data['onnx_model_uri']
train_job_id =  model_info_data['train_job_id']
prepare_kpi_dashboard = False


In [5]:
payload = {}
payload["onnx_model_uri"] = onnx_model_uri
payload["mas_device_name"] = mas_device_name
payload["train_job_id"] = train_job_id
payload['prepare_kpi_dashboard'] = prepare_kpi_dashboard

files = [
    ("iot_credentials", (credentials_file_name, open(credentials_file_path), "text/csv"))
]

<a id='deploy_model_post'></a> 
### Post Model Deployment Job 

In [7]:
endpoint_url = model_factory_config["endpoint_url"]
deploy_recipe_endpoint = endpoint_url + model_factory_config["deploy_recipe_endpoint"]
print(deploy_recipe_endpoint)

http://127.0.0.1:8000/ibm/modelfactory/service/deployment/monitor/model/create


In [8]:
headers = {
  'accept': 'application/json'
}
response = requests.post(deploy_recipe_endpoint, data=payload, files=files, headers=headers)

post_r_json = response.json()
print(post_r_json)
job_id = post_r_json['job_id']
print('job id:',job_id )

{'job_id': 'b5e1cb62-69ba-47e4-b1a6-23164ec090c9', 'message': 'Job b5e1cb62-69ba-47e4-b1a6-23164ec090c9 was submitted.', 'status': 'INITIALIZING'}
job id: b5e1cb62-69ba-47e4-b1a6-23164ec090c9


<a id='job_log'></a> 
### Request job log and summary

In [11]:
logs = requests.get(endpoint_url + "log/" + job_id, headers=headers)
print('LOGS :: ')
if "logs" in logs.json():
    print(logs.json()["logs"])
else:
    print(logs.json())

summary = requests.get(endpoint_url + "summary/" + job_id, headers=headers)
print('SUMMARY :: ')
print(summary.json())

LOGS :: 
recipe_file_path /tmp/ray/session_2023-11-20_20-43-06_133706_1/runtime_resources/working_dir_files/_ray_pkg_2d7293cea8265f5f/RecipeDeployONNX2IoT
entrypoint_path /tmp/ray/session_2023-11-20_20-43-06_133706_1/runtime_resources/working_dir_files/_ray_pkg_2d7293cea8265f5f/main.py
2023-11-21 01:52:05,943	INFO worker.py:1329 -- Using address 172.63.0.4:6379 set in the environment variable RAY_ADDRESS
2023-11-21 01:52:05,943	INFO worker.py:1458 -- Connecting to existing Ray cluster at address: 172.63.0.4:6379...
2023-11-21 01:52:05,971	INFO worker.py:1633 -- Connected to Ray cluster. View the dashboard at 172.63.0.4:8265 
(run_recipe_on_ray pid=1731) 2023/11/21 01:52:08 INFO mlflow.tracking.fluent: Experiment with name 'b5e1cb62-69ba-47e4-b1a6-23164ec090c9' does not exist. Creating a new experiment.
(run_recipe_on_ray pid=1731) mlflow.experiment.name: b5e1cb62-69ba-47e4-b1a6-23164ec090c9
(run_recipe_on_ray pid=1731) mlflow.experiment.id: 41
(run_recipe_on_ray pid=1731) 2023/11/21 01